# Model exporting
Sometimes your model runtime environment may be different your training environment, and this can sometimes involve changing tools, e.g. CMS has a Tensorflow interface built in to its main software package. If your runtime does not yet support PyTorch, you may need to export your trained models in a format that they can then be applied in production.
LUMIN currently has limited, experimental support for exporting to [ONNX](https://github.com/onnx/onnx) (open standard format for representing machine learning models), and [Tensorflow](https://www.tensorflow.org/) Protocolbuffer.

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import warnings
import pickle

In [2]:
from pathlib import Path
SAVE_PATH = Path('weights/')

We'll begin by loading a model that was trained during the Binary Classification example

In [3]:
from lumin.nn.models.model import Model

In [4]:
with open(SAVE_PATH/'Binary_Classification_builder.pkl', 'rb') as fin: model_builder = pickle.load(fin)

In [5]:
model = Model.from_save(SAVE_PATH/'Binary_Classification_0.h5', model_builder)

In [6]:
model

Model:
<bound method Module.parameters of Sequential(
  (0): CatEmbHead(
    (embeds): ModuleList(
      (0): Embedding(4, 2)
    )
  )
  (1): FullyConnected(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=32, out_features=100, bias=True)
        (1): Swish()
      )
      (1): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Swish()
      )
      (2): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Swish()
      )
      (3): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Swish()
      )
    )
  )
  (2): ClassRegMulti(
    (dense): Linear(in_features=100, out_features=1, bias=True)
    (act): Sigmoid()
  )
)>
                   

Number of trainable parameters: 33709
                   

Optimiser:
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0004272996042924875
    weight_decay

# ONNX

In order to export to ONNX, we need to hardcode a batch size for the data that will be fed through the model during runtime. Since in a physics analysis, data is normally processed serially, we'll set the batchsize to one. Note: Ensemble also has a `/export2onnx` method which will export all models.

In [7]:
model.export2onnx(str(SAVE_PATH/'Binary_Classification'), bs=1)

Now we can load the exported model to check

In [8]:
import onnx
onnx_model = onnx.load(SAVE_PATH/'Binary_Classification.onnx')
onnx.checker.check_model(onnx_model)

In [9]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch-jit-export (
  %0[FLOAT, 1x31]
) initializers (
  %0.embeds.0.weight[FLOAT, 4x2]
  %1.layers.0.0.weight[FLOAT, 100x32]
  %1.layers.0.0.bias[FLOAT, 100]
  %1.layers.1.0.weight[FLOAT, 100x100]
  %1.layers.1.0.bias[FLOAT, 100]
  %1.layers.2.0.weight[FLOAT, 100x100]
  %1.layers.2.0.bias[FLOAT, 100]
  %1.layers.3.0.weight[FLOAT, 100x100]
  %1.layers.3.0.bias[FLOAT, 100]
  %2.dense.weight[FLOAT, 1x100]
  %2.dense.bias[FLOAT, 1]
) {
  %12 = Slice[axes = [1], ends = [9223372036854775807], starts = [30]](%0)
  %13 = Cast[to = 7](%12)
  %14 = Constant[value = <Scalar Tensor []>]()
  %15 = Gather[axis = 1](%13, %14)
  %16 = Gather(%0.embeds.0.weight, %15)
  %17 = Concat[axis = 1](%16)
  %18 = Slice[axes = [1], ends = [30], starts = [0]](%0)
  %19 = Concat[axis = 1](%18, %17)
  %20 = Gemm[alpha = 1, beta = 1, transB = 1](%19, %1.layers.0.0.weight, %1.layers.0.0.bias)
  %21 = Sigmoid(%20)
  %22 = Mul(%20, %21)
  %23 = Gemm[alpha = 1, beta = 1, transB = 1](%22, %1.layers.1.0.weight, %1.l

And visualise it with netron to make sure it looks as expected

In [10]:
import netron
netron.start(str(SAVE_PATH/'Binary_Classification.onnx'))

Serving 'weights/Binary_Classification.onnx' at http://localhost:8080


# Tensorflow
Exporting to Tensorflow requires fir exporting to ONNX, then converting the ONNX file to a protocol buffer. Both exports can be performed using the `.export2tfpb` methods.

In [11]:
model.export2tfpb(str(SAVE_PATH/'Binary_Classification.pb'))

W0731 10:07:15.126302 139835358205760 deprecation_wrapper.py:119] From /home/giles/anaconda3/lib/python3.7/site-packages/onnx_tf/handlers/backend/ceil.py:10: The name tf.ceil is deprecated. Please use tf.math.ceil instead.

W0731 10:07:15.128237 139835358205760 deprecation_wrapper.py:119] From /home/giles/anaconda3/lib/python3.7/site-packages/onnx_tf/handlers/backend/depth_to_space.py:12: The name tf.depth_to_space is deprecated. Please use tf.compat.v1.depth_to_space instead.

W0731 10:07:15.129291 139835358205760 deprecation_wrapper.py:119] From /home/giles/anaconda3/lib/python3.7/site-packages/onnx_tf/handlers/backend/erf.py:9: The name tf.erf is deprecated. Please use tf.math.erf instead.

W0731 10:07:15.398193 139835358205760 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * ht

Manually, this involves running:

In [12]:
from onnx_tf.backend import prepare

In [13]:
tf_rep = prepare(onnx_model)

In [14]:
print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model

['0']
-----
['33']
-----
{'0.embeds.0.weight': <tf.Tensor 'Const:0' shape=(4, 2) dtype=float32>, '1.layers.0.0.bias': <tf.Tensor 'Const_1:0' shape=(100,) dtype=float32>, '1.layers.0.0.weight': <tf.Tensor 'Const_2:0' shape=(100, 32) dtype=float32>, '1.layers.1.0.bias': <tf.Tensor 'Const_3:0' shape=(100,) dtype=float32>, '1.layers.1.0.weight': <tf.Tensor 'Const_4:0' shape=(100, 100) dtype=float32>, '1.layers.2.0.bias': <tf.Tensor 'Const_5:0' shape=(100,) dtype=float32>, '1.layers.2.0.weight': <tf.Tensor 'Const_6:0' shape=(100, 100) dtype=float32>, '1.layers.3.0.bias': <tf.Tensor 'Const_7:0' shape=(100,) dtype=float32>, '1.layers.3.0.weight': <tf.Tensor 'Const_8:0' shape=(100, 100) dtype=float32>, '2.dense.bias': <tf.Tensor 'Const_9:0' shape=(1,) dtype=float32>, '2.dense.weight': <tf.Tensor 'Const_10:0' shape=(1, 100) dtype=float32>, '0': <tf.Tensor '0:0' shape=(1, 31) dtype=float32>, '12': <tf.Tensor 'Slice:0' shape=(1, 1) dtype=float32>, '13': <tf.Tensor 'Cast:0' shape=(1, 1) dtype=int6

In [15]:
tf_rep.export_graph(SAVE_PATH/'Binary_Classification.pb')